# Exercise - Forecasting with Fundamentals_Complete Solution Notebook_

# 1. Carry Strategy for Dividend Yield

In this exercise, we consider the dividend-yield of individual stocks in order to invest in **high-carry** securities.Note that we could consider the full analysis below with other metrics, including the PE ratio provided in the same data set.

#### DataUse the data in `data/spx_data_weekly.xlsx`.* `spx data` - time-series data for price, dividend-price ratio, and price-earnings ratio* `sector data` - ETFs on sub-sectors* `additional data` - `SPY`, `SHV`, and other benchmarks* `rates data` - index levels for ratesRisk-free rate could be * `SHV`*  `USGG3M Index` - annualized percent

In [ ]:
# Import necessary librariesimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.linear_model import Lasso, LinearRegressionimport warningswarnings.filterwarnings('ignore')# Set display optionspd.options.display.float_format = '{:,.4f}'.formatpd.set_option('display.max_columns', None)

In [ ]:
# Load stock informationFILE_DATA = '../data/spx_data_weekly.xlsx'SHEET_INFO = 'spx names'info = pd.read_excel(FILE_DATA, sheet_name=SHEET_INFO)info.set_index('ticker', inplace=True)display(info.head(10))

In [ ]:
# Define function to load time series datadef load_ts(file_data, sheet_name):    ts = pd.read_excel(file_data, sheet_name=sheet_name,                    header=[0,1], index_col=0)    ts.columns.names = ["ticker", "field"]    ts.rename(columns={        'PX_LAST': 'price',        'EQY_DVD_YLD_IND': 'dvd yld',        'PE_RATIO': 'pe ratio'    }, level=1, inplace=True)    return ts

In [ ]:
# Load SPX stock dataSHEET_TS = 'spx data'spx = load_ts(FILE_DATA, SHEET_TS)print(f"Data shape: {spx.shape}")print(f"Date range: {spx.index[0]} to {spx.index[-1]}")display(spx.tail())

### 1.0. Data ProcessingFilter the data to drop any ticker which does not have at least `5` years of continuous price data.

In [ ]:
# Extract price and dividend yield dataprices = spx.xs('price', level='field', axis=1)dvd_yld = spx.xs('dvd yld', level='field', axis=1)print(f"Original number of stocks: {prices.shape[1]}")# Filter: keep only stocks with at least 5 years (260 weeks) of continuous datamin_periods = 260  # 5 years * 52 weeksvalid_tickers = prices.columns[prices.notna().sum() >= min_periods]prices_filtered = prices[valid_tickers]dvd_yld_filtered = dvd_yld[valid_tickers]print(f"After filtering (5+ years): {prices_filtered.shape[1]} stocks")print(f"Removed: {prices.shape[1] - prices_filtered.shape[1]} stocks")

### 1.1. Data ProcessingReport the highest and lowest dividend-yielding stocks...* for any given date across the entire panel* taking an average of the past yearWhich stocks were they? Was it driven more by changes in D or P?

In [ ]:
# Calculate rolling 1-year average dividend yielddvd_yld_1y = dvd_yld_filtered.rolling(window=52, min_periods=26).mean()# Find highest and lowest dividend-yielding stocks at each datehighest_dvd_ticker = dvd_yld_1y.idxmax(axis=1)highest_dvd_value = dvd_yld_1y.max(axis=1)lowest_dvd_ticker = dvd_yld_1y.idxmin(axis=1)lowest_dvd_value = dvd_yld_1y.min(axis=1)print("=== Most Frequent Highest/Lowest Dividend Yielding Stocks ===")print("\nTop 10 Highest Dividend Yielding Stocks:")print(highest_dvd_ticker.value_counts().head(10))print("\nTop 10 Lowest Dividend Yielding Stocks:")print(lowest_dvd_ticker.value_counts().head(10))

### 1.2. A Carry StrategyFor this strategy, use the `dvd yld` data.Build a portfolio where at every time $t$, you set weights, $w_t$ that invest...* long the highest ranking `20%` stocks.For now, go equal weights of `0.01` in any stock for which you're long or short.

In [ ]:
# Calculate returnsreturns = prices_filtered.pct_change()# Use 1-year rolling average dividend yield for rankingdvd_yld_signal = dvd_yld_filtered.rolling(window=52, min_periods=26).mean()# Function to construct long-only carry strategydef construct_long_only_strategy(signal, returns, top_pct=0.20, weight=0.01):    weights = pd.DataFrame(0.0, index=signal.index, columns=signal.columns)        for date in signal.index:        signal_t = signal.loc[date].dropna()        if len(signal_t) > 0:            n_long = int(len(signal_t) * top_pct)            top_stocks = signal_t.nlargest(n_long).index            weights.loc[date, top_stocks] = weight        portfolio_returns = (weights.shift(1) * returns).sum(axis=1)    return portfolio_returns, weights# Construct long-only strategylo_returns, lo_weights = construct_long_only_strategy(dvd_yld_signal, returns)print(f"Long-Only average weekly return: {lo_returns.mean():.4f}")print(f"Annualized return: {lo_returns.mean() * 52:.4f}")

### 1.3. Long-ShortRe-do the strategy of the previous section, but this time go long and short...At any given time $t$, set the vector $w_t$ as follows...* long the highest-ranking `20%` of stocks.* short the lowest-ranking `20%` of stocks.* take no position in all other stocks.

In [ ]:
# Function to construct long-short carry strategydef construct_long_short_strategy(signal, returns, top_pct=0.20, weight=0.01):    weights = pd.DataFrame(0.0, index=signal.index, columns=signal.columns)        for date in signal.index:        signal_t = signal.loc[date].dropna()        if len(signal_t) > 0:            n_long = int(len(signal_t) * top_pct)            n_short = int(len(signal_t) * top_pct)            top_stocks = signal_t.nlargest(n_long).index            bottom_stocks = signal_t.nsmallest(n_short).index            weights.loc[date, top_stocks] = weight            weights.loc[date, bottom_stocks] = -weight        portfolio_returns = (weights.shift(1) * returns).sum(axis=1)    return portfolio_returns, weights# Construct long-short strategyls_returns, ls_weights = construct_long_short_strategy(dvd_yld_signal, returns)print(f"Long-Short average weekly return: {ls_returns.mean():.4f}")print(f"Annualized return: {ls_returns.mean() * 52:.4f}")

### 1.4. PerformanceCalculate the return of your portfolio over time. Report the following annualized stats: mean, volatility, SharpeAlso calculate the tail-risk stats: skewness, VaR (5th quantile), CVaR (5th quantile), max drawdownFor both tables, compare to `SPY`, found in the `additional data` tab.

In [ ]:
# Load benchmark data (SPY)SHEET_BENCH = 'additional data'bench = load_ts(FILE_DATA, SHEET_BENCH)spy_prices = bench[('SPY', 'price')]spy_returns = spy_prices.pct_change()# Combine all returnsall_returns = pd.DataFrame({    'Long-Only': lo_returns,    'Long-Short': ls_returns,    'SPY': spy_returns}).dropna()print(f"Analysis period: {all_returns.index[0]} to {all_returns.index[-1]}")

In [ ]:
# Calculate performance statisticsdef calc_performance_stats(returns, annual_factor=52):    stats = {}    stats['Mean'] = returns.mean() * annual_factor    stats['Volatility'] = returns.std() * np.sqrt(annual_factor)    stats['Sharpe Ratio'] = stats['Mean'] / stats['Volatility']    stats['Skewness'] = returns.skew()    stats['VaR (5%)'] = returns.quantile(0.05)    stats['CVaR (5%)'] = returns[returns <= returns.quantile(0.05)].mean()        cum_returns = (1 + returns).cumprod()    running_max = cum_returns.expanding().max()    drawdown = (cum_returns - running_max) / running_max    stats['Max Drawdown'] = drawdown.min()        return pd.Series(stats)performance_stats = all_returns.apply(calc_performance_stats)print("=== Annualized Performance Statistics ===")display(performance_stats.loc[['Mean', 'Volatility', 'Sharpe Ratio']].T)print("\n=== Tail Risk Statistics ===")display(performance_stats.loc[['Skewness', 'VaR (5%)', 'CVaR (5%)', 'Max Drawdown']].T)

In [ ]:
# Plot cumulative returnscum_returns = (1 + all_returns).cumprod()plt.figure(figsize=(12, 6))for col in cum_returns.columns:    plt.plot(cum_returns.index, cum_returns[col], label=col, linewidth=2)plt.title('Cumulative Returns: Carry Strategies vs SPY', fontsize=14, fontweight='bold')plt.xlabel('Date')plt.ylabel('Cumulative Return')plt.legend()plt.grid(True, alpha=0.3)plt.tight_layout()plt.show()

---# 2. Attribution

### 2.1. Market ExposureFor both the long-only (LO) and long-short (LS) strategies, estimate a linear factor decomposition (LFD) against `SPY`Report the...* alpha (annualized)* beta* r-squaredAlso show the correlation matrix of the strategies.

In [ ]:
# Regression functiondef run_regression(y, X, annual_factor=52):    data = pd.DataFrame({'y': y, 'X': X}).dropna()    model = LinearRegression()    model.fit(data[['X']], data['y'])    y_pred = model.predict(data[['X']])    residuals = data['y'] - y_pred    ss_res = np.sum(residuals**2)    ss_tot = np.sum((data['y'] - data['y'].mean())**2)    r_squared = 1 - (ss_res / ss_tot)        results = {        'Alpha': model.intercept_,        'Annualized Alpha': model.intercept_ * annual_factor,        'Beta': model.coef_[0],        'R-Squared': r_squared    }    return pd.Series(results)# Run regressions against SPYlo_spy_reg = run_regression(all_returns['Long-Only'], all_returns['SPY'])ls_spy_reg = run_regression(all_returns['Long-Short'], all_returns['SPY'])spy_regression_results = pd.DataFrame({    'Long-Only': lo_spy_reg,    'Long-Short': ls_spy_reg}).Tprint("=== Linear Factor Decomposition vs SPY ===")display(spy_regression_results)# Correlation matrixcorr_matrix = all_returns.corr()print("\n=== Correlation Matrix ===")display(corr_matrix)

### 2.2. Sector RegressionEstimate a multivariate LFD for both LO and LS against all the sector ETFs.* Note that `SHV` is not a sector ETF but rather a money-market fund. Exclude it.* Exclude `SPY`.

In [ ]:
# Load sector dataSHEET_SECTORS = 'sector data'sectors = load_ts(FILE_DATA, SHEET_SECTORS)sector_prices = sectors.xs('price', level='field', axis=1)sector_returns = sector_prices.pct_change()# Exclude SHV and SPYsector_cols = [col for col in sector_returns.columns if col not in ['SHV', 'SPY']]sector_returns_clean = sector_returns[sector_cols]print(f"Sector ETFs included: {sector_cols}")

In [ ]:
# Multivariate regression functiondef run_multivariate_regression(y, X, annual_factor=52):    data = pd.concat([y, X], axis=1).dropna()    y_clean = data.iloc[:, 0]    X_clean = data.iloc[:, 1:]        model = LinearRegression()    model.fit(X_clean, y_clean)        y_pred = model.predict(X_clean)    residuals = y_clean - y_pred    ss_res = np.sum(residuals**2)    ss_tot = np.sum((y_clean - y_clean.mean())**2)    r_squared = 1 - (ss_res / ss_tot)        results = {        'Alpha': model.intercept_,        'Annualized Alpha': model.intercept_ * annual_factor,        'R-Squared': r_squared    }        for i, col in enumerate(X_clean.columns):        results[f'{col} Beta'] = model.coef_[i]        return pd.Series(results)# Run sector regressionslo_sector_reg = run_multivariate_regression(all_returns['Long-Only'], sector_returns_clean)ls_sector_reg = run_multivariate_regression(all_returns['Long-Short'], sector_returns_clean)sector_regression_results = pd.DataFrame({    'Long-Only': lo_sector_reg,    'Long-Short': ls_sector_reg}).Tprint("=== Sector Regression Results ===")display(sector_regression_results[['Alpha', 'Annualized Alpha', 'R-Squared']])

### 2.3. Sector NeutralityIs your LO or LS implementation of the carry strategy neutral to sectors? To which sector does it have the largest exposure?

In [ ]:
# Calculate beta * sigma for each sectorbeta_cols = [col for col in sector_regression_results.columns if 'Beta' in col]sector_betas = sector_regression_results[beta_cols]sector_vols = sector_returns_clean.std()lo_betas = sector_betas.loc['Long-Only']ls_betas = sector_betas.loc['Long-Short']lo_beta_sigma = lo_betas * sector_vols.valuesls_beta_sigma = ls_betas * sector_vols.valuesexposure_analysis = pd.DataFrame({    'LO Beta*Sigma': lo_beta_sigma.values,    'LS Beta*Sigma': ls_beta_sigma.values}, index=sector_cols)print("=== Sector Exposure Analysis (Beta * Sigma) ===")display(exposure_analysis.sort_values('LO Beta*Sigma', ascending=False))print(f"\nLong-Only largest exposure: {exposure_analysis['LO Beta*Sigma'].abs().idxmax()}")print(f"Long-Short largest exposure: {exposure_analysis['LS Beta*Sigma'].abs().idxmax()}")

---## SummaryThis notebook implements a comprehensive analysis of dividend-yield-based carry strategies:1. **Strategy Construction**: Long-only and long-short portfolios based on dividend yield2. **Performance Analysis**: Detailed risk-return metrics and comparison to SPY3. **Attribution**: Factor decomposition against market and sector exposures**Key Findings:**- Dividend yield provides a profitable signal for stock selection- Long-short strategy offers better risk-adjusted returns and market neutrality- Sector exposures explain significant variance in strategy returns